<a href="https://colab.research.google.com/github/gibiee/study_AI/blob/master/Machine%20Learning/3_Softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [0]:
nb_classes = 3

W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

In [0]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

    return grads

In [43]:
optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

epochs = 1000
for i in range(epochs):
    grads = grad_fn(x_data, y_data)
    optimizer.apply_gradients(zip(grads, variables))
    if i == 0 or (i+1) % 100 == 0:
        print('Loss at epoch %d: %f' %(i+1, cost_fn(x_data, y_data).numpy()))

Loss at epoch 1: 3.154977
Loss at epoch 100: 0.643107
Loss at epoch 200: 0.577626
Loss at epoch 300: 0.528178
Loss at epoch 400: 0.485983
Loss at epoch 500: 0.447827
Loss at epoch 600: 0.412047
Loss at epoch 700: 0.377461
Loss at epoch 800: 0.343054
Loss at epoch 900: 0.307969
Loss at epoch 1000: 0.272325


In [44]:
# x_data를 입력으로 하여 y를 예측
predict_y = hypothesis(x_data)
print(predict_y)
print(tf.argmax(predict_y, 1))  # 예측값
print(tf.argmax(y_data, 1)) # 실제 정답값

tf.Tensor(
[[2.8888084e-05 7.8096115e-03 9.9216145e-01]
 [2.5788412e-04 1.3716571e-01 8.6257637e-01]
 [1.8078650e-05 2.8598225e-01 7.1399969e-01]
 [2.9589051e-05 8.1423968e-01 1.8573070e-01]
 [2.8631273e-01 6.9539034e-01 1.8296819e-02]
 [1.4595546e-01 8.5379475e-01 2.4976427e-04]
 [5.0322771e-01 4.9637780e-01 3.9456342e-04]
 [7.6150030e-01 2.3848020e-01 1.9510186e-05]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


# **클래스로 구현하여 간단하게 동작시키기**

In [45]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 3.808221
Loss at epoch 500: 0.471691
Loss at epoch 1000: 0.288607
Loss at epoch 1500: 0.205475
Loss at epoch 2000: 0.167571
